# Install Library yang diperlukan

In [ ]:
!pip install pdfplumber

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.1/46.1 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 44.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 73.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.6/143.6 kB 15.6 MB/s eta 0:00:00
  Attempting uninstall: Pillow
    Found existing installation: Pillow 8.4.0
    Uninstalling Pillow-8.4.0:
      Successfully uninstalled Pillow-8.4.0


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/_Tetris')

Ini adalah script untuk mengubah pdf lampiran hasil CPNS jadi csv
Caveat: kalo langsung prosess 20rb halaman, agak keselek di tengah, jadi mending di-iterate beberapa halaman
Saya yakin ada cara yang lebih elegan. Feel free to give me any suggestions

Cara run </br>
python export_data_to_csv.py index_halaman_start index_halaman_end

Misal </br>
python export_data_to_csv.py 0 100

In [ ]:
# import library
import pdfplumber
import pandas as pd
import datetime as dt
import sys

# Data Tahap 1

In [ ]:
def check_for_jabatan(page):
    '''
    Fungsi untuk mengecek keberadaan informasi jurusan pilihan
    Input: page (ex: pdf.pages[0])
    Output: 
        - dicitionary berisi informasi jurusan pilihan
    '''
    if "Lokasi Formasi :" in page.extract_text():
        text = pg.extract_text()
        pendidikan = pg.extract_tables()[1][0][1]
        jurusan_strings = text.split("Jurusan : ")[1].split("Lokasi")[0]
        lokasi_front = ""
        jurusan = jurusan_strings.split("\n")[0]

        lokasi_string = text.split("Lokasi Formasi : ")[1].split("Jenis")[0]

        return {
            "kode_jurusan": jurusan.split(" - ")[0],
            "jurusan": jurusan.split(" - ")[1],
            "kode_lokasi": lokasi_string.split(" - ")[0],
            "lokasi_formasi": lokasi_string.split(" - ")[1].split("\n")[0],
            "pendidikan": pendidikan.replace("\n", " ")
        }
    else:
        return {}

def check_for_detail_tables(page):
    '''
    Fungsi untuk mengecek keberadaan tabel perorangan
    Input: page (ex: pdf.pages[0])
    Output: 
        - found (binary, iya tidaknya sebuah halaman punya tabel perorangan)
        - df_returned (tabel perorangan jika ada)
    '''
    found = False
    df_returned = pd.DataFrame()
    for table in pg.extract_tables():
        df = pd.DataFrame(table)

        if (df.shape[1] == 9): #jumlah kolom = 9 
            found = True
            df_returned = df
    return found, df_returned


def split_df(df_):
    '''
    Fungsi untuk split tabel perorangan. Kadang ada satu halaman dengan lebih dari satu tabel.
    Input: df_ (dataframe tabel perorangan)
    Output: 
        - list berisi beberapa dataframe untuk tiap individu
    '''
    dfs = []
    header_indexes = list(df_[df_[1]=="No Peserta"].index)
    header_indexes.append(len(df_))
    for i in range(len(header_indexes)-1):
        splitted_df = df_.iloc[header_indexes[i]: header_indexes[i+1], :]
        splitted_df.index = range(len(splitted_df))
        dfs.append(splitted_df)
    return dfs

def get_info_from_table(df_, idx):
    '''
    Fungsi untuk mengekstrak informasi dari tabel perorangan 
    Input: df_ (dataframe tabel perorangan)
    Output: 
        - dicitionary berisi informasi perorangan yang telah diekstrak
    '''
    base_data =  {
        "no_peserta": df_.iloc[idx,1],
        "kode_pendidikan": df_.iloc[idx,2],
        "total_twk": df_.iloc[idx,4],
        "total_tiu": df_.iloc[idx,5],
        "total_tkp": df_.iloc[idx,6],
        "total_skd": df_.iloc[idx,7],
        "keterangan": df_.iloc[idx,8]
    }

    return {**base_data}

In [ ]:
start_index = 0
end_index = 929
export_filename = "spmb_"+str(start_index)+"_"+str(end_index)+"_"+str(dt.datetime.now())+".csv"
pdf = pdfplumber.open('datasets/Lampiran B-099.02720.06.2022 Pengumuman Hasil Tahap 1 Tahun 2022.pdf')

# start iterating
result = []
last_jabatan = {}

start_time = dt.datetime.now()
for i in range(start_index, end_index):
    pg = pdf.pages[i]

    # jika halaman tsb ada info tentang lowongan, simpan
    current_jabatan = check_for_jabatan(pg)
    if current_jabatan != {}:
        last_jabatan = current_jabatan
    
    is_detail_found, detail_df = check_for_detail_tables(pg)
    # jika ketemu ada tabel perorangan
    if is_detail_found:
        splitted_df = split_df(detail_df)
        for df_ in splitted_df:
            for i in range(2, len(df_)):
                details = get_info_from_table(df_, i)
                if current_jabatan == {}:
                    # kalo ada info lowongan di halaman yang sama, pakai info lowongan tsb
                    details.update(last_jabatan)
                else:
                    # kalo ga, pake info lowongan terakhir
                    details.update(current_jabatan)
                    last_jabatan = current_jabatan
                result.append(details)

    # untuk logging
    if i%100 == 99:
        curr_time = dt.datetime.now()
        print("Done for "+str(i), curr_time-start_time)
        start_time = dt.datetime.now()

res = pd.DataFrame(result)
res.to_csv(export_filename)